In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
import matplotlib.pyplot as plt# Train the model on training data
from preProcessing import PreProcessing
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
import pathlib
from preProcessing import PreProcessing
from PIL import Image
import numpy as np 
from numpy import asarray

train_images = []
train_labels = []

path = pathlib.Path("." + "/all-images/train")
for class_name in path.iterdir():
    label = class_name.name
    path = pathlib.Path("." + "/all-images/train"+ f"/{label}")
    for im in path.iterdir():
        im = Image.open(im)
        obj = PreProcessing(im)
        final_image = obj.pre_processed()
        final_image = final_image.resize((128, 128))
        train_images.append(asarray(final_image))
        train_labels.append(label)
       
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
print(f"do::::::::{len(train_images)}") 
final_image.size

In [ ]:
test_images = []
test_labels = []

path = pathlib.Path("." + "/all-images/validation")
for class_name in path.iterdir():
    label = class_name.name
    path = pathlib.Path("." + "/all-images/validation"+ f"/{label}")
    for im in path.iterdir():
        im = Image.open(im)
        obj = PreProcessing(im)
        final_image = obj.pre_processed()
        final_image = final_image.resize((128, 128))
        test_images.append(asarray(final_image))
        test_labels.append(label)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
print(f"do::::::::{len(test_images)}") 
final_image.size

In [ ]:
#encode labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
len(test_labels_encoded)

In [ ]:
#rename variables
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

#scalling pixel values  to between 0 and 1
x_train, x_test = x_train/255.0, x_test/255.0 

#one hot encode y vars for neural network
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

#reshape x_train from 3 dims to 4 dims
x_train = x_train.reshape(1058, 128, 128, 1)
#reshape x_test from 3 dims to 4 dims
x_test = x_test.reshape(454, 128, 128, 1)

In [ ]:
len(y_train_one_hot)

In [ ]:
#neural network model 

feature_extractor = Sequential()

feature_extractor.add(Conv2D(16, kernel_size=(6,6), padding = 'same', input_shape = (128, 128, 1)))
feature_extractor.add(LeakyReLU(alpha=0.1))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D(pool_size=(2, 2), strides=2))
feature_extractor.add(Dropout(0.5))

feature_extractor.add(Conv2D(16, kernel_size=(5,5), padding = 'same', kernel_initializer = 'he_normal'))
feature_extractor.add(LeakyReLU(alpha=0.1))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D(pool_size=(2, 2), strides=2))
feature_extractor.add(Dropout(0.5))

feature_extractor.add(Conv2D(16, kernel_size=(3,3), padding = 'same', kernel_initializer = 'he_normal'))
feature_extractor.add(LeakyReLU(alpha=0.1))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D(pool_size=(2, 2), strides=2))
feature_extractor.add(Dropout(0.5))

feature_extractor.add(Flatten())

In [ ]:
#add deep learning layers
x = feature_extractor.output

x = Dense(128, kernel_initializer = 'he_normal')(x)
x = (LeakyReLU(alpha=0.1))(x)
x = (Dropout(0.5))(x)

x = Dense(128, kernel_initializer = 'he_normal')(x)
x = (LeakyReLU(alpha=0.1))(x)
x = (Dropout(0.5))(x)


x = Dense(128, kernel_initializer = 'he_normal')(x)
x = (LeakyReLU(alpha=0.1))(x)
x = (Dropout(0.5))(x)


prediction_layer = Dense(2, activation = 'softmax')(x)

In [ ]:
#compile model
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])
print(cnn_model.summary()) 

In [ ]:
#Train the CNN model
history = cnn_model.fit(x_train, y_train_one_hot, epochs=100, validation_data = (x_test, y_test_one_hot))

In [ ]:
plt.figure(1)

# summarize history for accuracy

plt.subplot(211)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')

# summarize history for loss

plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

plt.tight_layout()

plt.show()

In [ ]:
prediction_NN = cnn_model.predict(x_test)
prediction_NN = np.argmax(prediction_NN, axis=-1)
prediction_NN = le.inverse_transform(prediction_NN)

In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(test_labels, prediction_NN)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)

In [ ]:
################################
#Now, use features from convolutional network for RF
X_for_RF = feature_extractor.predict(x_train) #This is out X input to RF

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 200)

In [ ]:
# Train the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)

In [ ]:
#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_feature)

In [ ]:
#Inverse le transform to get original label back. 
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

In [ ]:
#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(test_labels, prediction_RF)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)

In [ ]:
#Now, use features from convolutional network for knn
X_for_knn = feature_extractor.predict(x_train)

In [ ]:
################
#knn
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Train the model on training data
knn_model.fit(X_for_knn, y_train)

In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)

In [ ]:
#Now predict using the trained knn model. 
prediction_knn = knn_model.predict(X_test_feature)

In [ ]:
#Inverse le transform to get original label back. 
prediction_knn = le.inverse_transform(prediction_knn)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_knn))

In [ ]:
#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(test_labels, prediction_knn)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)

In [ ]:
#Now, use features from convolutional network for adaboost
X_for_adaboost = feature_extractor.predict(x_train)

In [ ]:
##############
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
adaboost_model = AdaBoostClassifier(n_estimators = 100)

In [ ]:
# Train the model on training data
adaboost_model.fit(X_for_adaboost, y_train)

In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)

In [ ]:
#Now predict using the trained adaboost model. 
prediction_adaboost = adaboost_model.predict(X_test_feature)

In [ ]:
#Inverse le transform to get original label back. 
prediction_adaboost = le.inverse_transform(prediction_adaboost)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_adaboost))

In [ ]:
#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(test_labels, prediction_adaboost)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)

In [ ]:
#Now, use features from convolutional network for DecisionTreeClassifier-dtc
X_for_dtc = feature_extractor.predict(x_train)

In [ ]:
##############
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtc_model = DecisionTreeClassifier(random_state = 0)

In [ ]:
# Train the model on training data
dtc_model.fit(X_for_dtc, y_train)

In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)

In [ ]:
#Now predict using the trained dtc model. 
prediction_dtc = dtc_model.predict(X_test_feature)

In [ ]:
#Inverse le transform to get original label back. 
prediction_dtc = le.inverse_transform(prediction_dtc)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_dtc))

In [ ]:
#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(test_labels, prediction_dtc)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)

In [ ]:
#Now, use features from convolutional network for LogisticRegression-lg
X_for_lg = feature_extractor.predict(x_train)

In [ ]:
##############
#LogisticRegression
from sklearn.linear_model import LogisticRegression
lg_model = LogisticRegression(random_state = 0)

In [ ]:
# Train the model on training data
lg_model.fit(X_for_lg, y_train)

In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)

In [ ]:
#Now predict using the trained lg model. 
prediction_lg = lg_model.predict(X_test_feature)

In [ ]:
#Inverse le transform to get original label back. 
prediction_lg = le.inverse_transform(prediction_lg)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_lg))

In [ ]:
#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(test_labels, prediction_lg)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)

In [ ]:
#Now, use features from convolutional network for SupportVectorMachine-svm
X_for_svm = feature_extractor.predict(x_train)

In [ ]:
##############
#SupportVectorMachine
from sklearn import svm
svm_model = svm.SVC()

In [ ]:
# Train the model on training data
svm_model.fit(X_for_svm, y_train)

In [ ]:
#Send test data through same feature extractor process
X_test_feature = feature_extractor.predict(x_test)

In [ ]:
#Now predict using the trained svm model. 
prediction_svm = svm_model.predict(X_test_feature)

In [ ]:
#Inverse le transform to get original label back. 
prediction_svm = le.inverse_transform(prediction_svm)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_svm))

In [ ]:
#Confusion Matrix - verify accuracy of each class
cm = confusion_matrix(test_labels, prediction_svm)
print(cm)
sns.heatmap(cm, annot=True)

total=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy=(cm[0,0]+cm[1,1])/total
print ('Accuracy : ', accuracy)

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity)

specificity = cm[1,1]/(cm[0,1]+cm[1,1])
print('Specificity : ', specificity)